In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
from urllib.request import urlretrieve
import libsbml
import amici
import os
from biosimulator_processes.utils import prepare_single_ode_process_document
from biosimulator_processes.data_model.compare_data_model import ODEComparisonDocument, DocumentFactory
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
_CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess registered successfully.
GetSbmlStep registered successfully.
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [2]:
import numpy as np 

y = np.array([0.5, 1.0, 0.75, 0.60, 0.66])

average_y = np.mean(y)

# Calculate the MSE for each value in y with respect to the average
mse_scores = (y - average_y) ** 2

mse_scores, average_y

(array([0.040804, 0.088804, 0.002304, 0.010404, 0.001764]), 0.7020000000000001)

In [3]:
# Step 1: Define the input parameters of the api method

biomodel_id = 'BIOMD0000000630'
sbml_dirpath = '../biosimulator_processes/model_files/sbml'
model_fp = os.path.join(sbml_dirpath, f'{biomodel_id}_url.xml')
species_context = 'concentrations'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30
n_steps = 42
simulators = ['copasi', 'tellurium']

In [4]:
# Step 2: Define a simple function that encapsulates that which is defined in the rest of the workflow

def generate_ode_verification(biomodel_id, dur) -> dict:
    compare = {
        'compare_ode': {
                '_type': 'step',
                  'address': 'local:compare_ode_step',
                  'config': {'biomodel_id': biomodel_id, 'duration': dur},
                  'inputs': {},
                  'outputs': {
                    'comparison_data': ['comparison_store']
                }
        },
        'verification_data': {
                 '_type': 'step',
                  'address': 'local:ram-emitter',
                  'config': {
                      'emit': {
                          'comparison_data': 'tree[any]'
                      }
                  },
                  'inputs': {
                      'comparison_data': ['comparison_store']
                  }
        }
    }
    
    
    wf = Composite(config={'state': compare}, core=CORE)
    wf.run(1)
    comparison_results = wf.gather_results()
    return comparison_results, wf

In [5]:
# Step 3: View the results. TODO: Add RMSE here.

verification_results, wf = generate_ode_verification(biomodel_id, duration)

pp(verification_results)

2024-05-13 13:56:06.943 - amici.sbml_import - INFO - Finished importing SBML                         (2.80E-02s)
2024-05-13 13:56:06.967 - amici.sbml_import - INFO - Finished processing SBML observables            (2.12E-02s)
2024-05-13 13:56:06.970 - amici.sbml_import - INFO - Finished processing SBML event observables      (6.66E-07s)
2024-05-13 13:56:06.992 - amici.de_model - INFO - Finished computing xdot                            (2.21E-03s)
2024-05-13 13:56:06.997 - amici.de_model - INFO - Finished computing x0                              (1.67E-03s)
2024-05-13 13:56:07.004 - amici.de_model - INFO - Finished computing w                               (5.12E-03s)


found a filepath


2024-05-13 13:56:07.385 - amici.de_export - INFO - Finished generating cpp code                      (3.77E-01s)
2024-05-13 13:56:16.524 - amici.de_export - INFO - Finished compiling cpp code                       (9.14E+00s)
/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/processes/copasi_process.py:205: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



{ ('verification_data',): [ { 'comparison_data': { ('emitter',): [ { 'amici_simple_floating_species_concentrations': { 'plasmin': 0.0,
                                                                                                                       'plasminogen': 0.0,
                                                                                                                       'single intact chain urokinase-type plasminogen activator': 0.0,
                                                                                                                       'two-chain urokinase-type plasminogen activator': 0.0,
                                                                                                                       'x': 0.0,
                                                                                                                       'x-plasmin': 0.0},
                                                                     'copasi_simple_floating_species_

In [6]:
verification_results

{('verification_data',): [{'comparison_data': {('emitter',): [{'copasi_simple_floating_species_concentrations': {'plasminogen': 0.0,
       'plasmin': 0.0,
       'single intact chain urokinase-type plasminogen activator': 0.0,
       'two-chain urokinase-type plasminogen activator': 0.0,
       'x': 0.0,
       'x-plasmin': 0.0},
      'amici_simple_floating_species_concentrations': {'plasminogen': 0.0,
       'plasmin': 0.0,
       'single intact chain urokinase-type plasminogen activator': 0.0,
       'two-chain urokinase-type plasminogen activator': 0.0,
       'x': 0.0,
       'x-plasmin': 0.0},
      'tellurium_simple_floating_species_concentrations': {'species_1': 0.0,
       'species_2': 0.0,
       'species_3': 0.0,
       'species_4': 0.0},
      'time': 0.0},
     {'copasi_simple_floating_species_concentrations': {'plasminogen': 1.1758171177387002e+16,
       'plasmin': 1096150505274.1506,
       'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,

In [7]:
"""TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:

{('emitter',): [{
    'copasi': [
        {
            'floating_species_concentrations': {
                'plasminogen': 0.0,
                'plasmin': 0.0,
                'single intact chain urokinase-type plasminogen activator': 0.0,
                'two-chain urokinase-type plasminogen activator': 0.0,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0
        },
        ...
    ],
    'amici': [
        {
            'floating_species_concentrations': {
                'plasminogen': 1.1758171177387002e+16,
                'plasmin': 1096150505274.1506,
                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,
                'two-chain urokinase-type plasminogen activator': 80249.33829510311,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0},
        },
       ...
    ]


"""

"TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:\n\n{('emitter',): [{\n    'copasi': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 0.0,\n                'plasmin': 0.0,\n                'single intact chain urokinase-type plasminogen activator': 0.0,\n                'two-chain urokinase-type plasminogen activator': 0.0,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0\n        },\n        ...\n    ],\n    'amici': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 1.1758171177387002e+16,\n                'plasmin': 1096150505274.1506,\n                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,\n                'two-chain urokinase-type plasminogen activator': 80249.33829510311,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0},\n        },\n 